# Defect supercell calculation setup for FHI-aims (adaptable for vasp)

A workflow for setting up supercell geometry files and calculation inputs for defect calculations with either vasp or FHI-aims (currently only for vacancies or antisites).

Transformer libraries by J. M. Skelton (See: https://github.com/JMSkelton).

Workflow and scripts by S. K. Wallace (email: s.k.wallace@bath.ac.uk or suzywallace501@gmail.com).

## 1. Get yourself setup with Transformer python libraries!
To use this setup, you just need to install the Transformer libraries, as explained [here](https://github.com/JMSkelton/Transformer) (NumPy and spglib libraries are needed) and then use the python scripts described in the next sections. Make sure you have the most up to date version of NumPy, if not, use '`sudo pip install --upgrade numpy`'. 

To see a progress bar during calculations, you can install the tqdm module using '`sudo pip install tqdm`', but usually calculating point defects doesn't take long enough for this to be required. 
You can also '`pip install cython`' to avoid getting a compilation error message for not having this, but we actually don't use cython functionality here so it isn't necessary to run this workflow. 

The (benign) error message you get for not having cython installed looks like this:

fatal error: stdarg.h: No such file or directory
 include <stdarg.h>
                    ^
compilation terminated.

**Under construction:** Look into flag to not use Cython to remove this error message

## 2. Generate a supercell from your relaxed unit cell
- Relax the volume of your unit cell using the same functional and species defaults (e.g. 'tight') as you intend to use for your defect supercell. 
- Add the following keywords to your control.in '`relax_geometry bfgs 5.e-3`' and '`relax_unit_cell full`', but do also refer to the FHI-aims manual under 'Tag: relax_geometry'. 
- Ensure that you use calculation settings (species defaults and kgrid density) for converged total energies.
- Add the geometry file for your relaxed unit cell to the same directory as this ipython notebook.
- Construct a supercell from this unit cell by running the script below (click on the cell, shift+enter), where you need to add the name of the geometry file for your relaxed unit cell, the format (i.e. vasp or aims) and the dimensions of your supercell.
- Aim for the largest and most isotropic (most similar size for all three lattice vectors) supercell that is computationally feasible. 
- You will then want to relax the geometry of this supercell, but for this do no relax the unit cell volume again, comment out this keyword in your control.in: '`#relax_unit_cell full`'. Some procedures relax the supercell volume also, but that is not the procedure adopted here.
- Note that as you increase the size of your system when constructing a supercell, your kgrid density scales inversely. For example, if you used a 4x4x4 kgrid for the unit cell, for a 2x2x2 supercell of this unit cell you can expect the same convergence for a 2x2x2 kgrid. 
- Depending on the size of your system it may only be possible to perform the geometry relaxation at the gamma point (1x1x1 kgrid), but later you will want to perform a single calculation with a denser kgrid. For this, you also comment out the keyword '`#relax_geometry bfgs 5.e-3`', and perform a single calculation using your optimised geometry.

### Transformer_supercell.py
(Add user inputs to the script below, shift+enter to run)

In [ ]:
# Routines from Transformer
from Transformer.IO.AIMS import ReadAIMSGeometryFile;
from Transformer.IO.VASP import ReadPOSCARFile;
from Transformer.IO.AIMS import WriteAIMSGeometryFile;
from Transformer.IO.VASP import WritePOSCARFile;
from Transformer.Constants import AtomicNumberToSymbol;

### USER INPUTS ##############################################################################
geom_file = "geometry.in" # Name of the geometry file for your relaxed unit cell
geom_format = "aims" # or vasp, if using a POSCAR
xdim = 2 # Dimensions of the supercell you wish to construct
ydim = 2 # Check against the lattice vectors for your input and aim for an isotropic supercell
zdim = 2 # (most similar x,y,z dimensions for your supercell as possible)
##############################################################################################

if (geom_format == "aims"):
  structure = ReadAIMSGeometryFile(geom_file);
else:
  structure = ReadPOSCARFile(geom_file);

supercell = structure.GetSupercell((xdim, ydim, zdim));
if (geom_format == "aims"):
  supercell_file = WriteAIMSGeometryFile(supercell, "supercell_geometry.in", None);
else:
  supercell_file = WritePOSCARFile(supercell, "supercell_POSCAR", None);

## 3. Check that Transformer recognises the spacegroup of your relaxed perfect supercell

This step allows you to check that spglib (libraries used by Transformer) recognises the correct space group for your input structure. Transformer recognises inequivalent defects based on the symmetry of the crystal structure, so if you have a relaxed input geometry, it is important that Transformer still picks up the correct space group for the structure. It may be necessary to adjust the tolerance after relaxing your structure until the correct space group has been identified. If the tolerance needed to recognise the correct symmetry is less than the default of this script (1.0e-5), **then this must be modified in the subsequent script when generating your defect supercells next**. As a general rule 1.0e-3 would be fairly tight, but 1.0e-2 would be a loose tolerance.
This first script also outputs the number of inequivalent species of each kind identified by Transformer with the selected tolerance. This is useful to know for the next step since, for example, the number of unique sites of Cu would tell us how many something-on-Cu antisite defects or Cu vacancies to expect for the structure.

To run the script below, you need to add to name of your supercell geometry file, the format (aims or vasp) and the spacegroup your structure should have. 
The output will check the space group against the number you enter and then print the number of each symmetry equivalent instance of each species, where the degeneracy is given in brackets. If your output is a P1 spacegroup and all sites are inequivalent then the likelihood is Transformer just hasn't recognised the symmetry, so try modifying the tolerance until the correct spacegroup is recognised, noting that 1.0e-3 would be fairly tight, but 1.0e-2 would be a loose.

### check_structure.py 
(Add user inputs to the script below, shift+enter to run)

In [ ]:
# Routines from Transformer
from Transformer.IO.AIMS import ReadAIMSGeometryFile;
from Transformer.IO.VASP import ReadPOSCARFile;
from Transformer.Framework.BatchIO import ExportResultSet;
from Transformer.Framework.Core import AtomicSubstitutions;
from Transformer.Constants import AtomicNumberToSymbol;

### USER INPUTS ##############################################################################
geom_file = "supercell_geometry.in"
geom_format = "aims"
spacegroup = 31
tolerance = 1.0e-5
##############################################################################################

# ----------------------------------------------------------------------------------------------------------
# Read input geometry file and check symmetry of input file is recognised by spglib
# ----------------------------------------------------------------------------------------------------------

if (geom_format == "aims"):
  structure = ReadAIMSGeometryFile(geom_file);
else:
  structure = ReadPOSCARFile(geom_file);

sgNumber, sgString = structure.GetSpacegroup(tolerance = tolerance);

print("");

# If spacegroup is not default value of 1000, assume user wishes to check the symmetry for the spacegroup they entered
if (spacegroup != 1000):
  if (sgNumber != spacegroup):
    print("Outcome of your symmetry check...");
    print("Oh dear, spglib has recognised {0} as the spacegroup of your input structure, but you entered {1}. Maybe you should double check your structure? Or try reducing the tolerance a little using '-t'. Maybe 5.0e-5?".format(sgNumber, spacegroup));
  else:
    print("Outcome of your symmetry check...");
    print("Good news, spglib has recognised your input structure spacegroup as {0}, you're good to go!".format(spacegroup));

# ----------------------------------------------------------------------------------------------------------
# Outputting number of inequivalent sites for each species
# ----------------------------------------------------------------------------------------------------------

print("");

indices, counts = structure.GetUniqueAtomIndices(tolerance);
atomTypeNumbers = structure.GetAtomTypeNumbers();

print("Unique atoms (site degeneracy):");

for index, count in zip(indices, counts):
    print("{0: >2} ({1: >2})".format(AtomicNumberToSymbol(atomTypeNumbers[index]), count));

## 4. Add to the same directory as this ipython notebook: a submission script for your cluster with 'JobNamePlaceholder' as the job title and a control.in file for your calculation with the keyword '#charge ChargePlaceholder' commented out at this stage
Example inputs can be found in this directory and this workflow could be easily adapted for vasp calculations by replacing instances of 'geometry.in' with POSCAR and 'control.in' with POTCAR. However, specifying the charge with NELECT is a little less of a direct substitution (but still not too complicated), but I have not explicitly made that an option here.

## 5. Generate charge neutral antisites and vacancies (interstitials not available yet)
Below is a python script for generating defect supercells using symmetry. The user needs to enter the name of the perfect supercell geometry file, the format (vasp or aims) the tolerance needed to recognise the correct space group of the structure in step 3 and then the defects you wish to make! Comment lines in and out as you see fit. e.g. set vacancy to `"no_vacancy"` if you want to generate antisites. For generating antisites, if you want to generate for example and As-on-Cu defect `antisite_in` would be `"As"` and `antisite_out` would be `"Cu"`

### Outputs
The script below will generate defects in directories organised as: VacancySupercells (or AntisiteSupercells) --> defect type and supercell composition in the name --> spacegroup of defective supercell, geometry.in for the neutral defect. In the next step, there is a bash script to extract and organise the outputs further.

** Under construction **

It will also produce a **defect_info.txt file** in each final defect directory telling you which line number in the supercell geometry file corresponds to your defect (in relation to perfect reference?). This is nice to know for visualising your structure and looking at geometry distortion after relaxation, but it is vital for later finite-size corrections to these defect supercell calculations.

### generate_neutral_defects.py
(Add user inputs to the script below, shift+enter to run)

In [ ]:
### Outputs:
# The code will produce a directory containing either your vacancy or antisite defects, in a tar format, with sub directories for structures with different space groups
# The first tar file (the one containing 'DefectType_001_...' is always the perfect supercell, then each subsequent are all possible defective supercells for each new lower symmetry space group

# Routines from Transformer
from Transformer.IO.AIMS import ReadAIMSGeometryFile;
from Transformer.IO.VASP import ReadPOSCARFile;
from Transformer.Framework.BatchIO import ExportResultSet;
from Transformer.Framework.Core import AtomicSubstitutions;
from Transformer.Constants import AtomicNumberToSymbol;

### USER INPUTS ##############################################################################
geom_file = "supercell_geometry.in"
geom_format = "aims"
tolerance = 1.0e-5 # Ensure this matches the tolerance needed to recognise the symmetry in step 3
vacancy = "Cu"
#vacancy = "no_vacancy"  # If generating antisites, set this to "no_vacancy"
#antisite_in = "As"
#antisite_out = "Cu"
antisite_in = "no_antisite" # If generating vacancies, set this to "no_antisite"
antisite_out = "no_antisite" # If generating vacancies, set this to "no_antisite"
##############################################################################################

# Reading in supercell geometry file
if (geom_format == "aims"):
  supercell = ReadAIMSGeometryFile(geom_file);
else:
  supercell = ReadPOSCARFile(geom_file);

# Generating vacancies if requested by user
if (vacancy != "no_vacancy"):
  substitutions = [(vacancy, None)];
  # Use the AtomicSubstitutions convenience function to generate the defective structures.
  _, resultSet = AtomicSubstitutions(
    supercell, substitutions, tolerance = tolerance
    );
  # Export the results.
#  ExportAtomicSubstitutionResultSet(
#      resultSet, prefix = "V-{0}".format(vacancy), workingDirectory = r"VacancySupercells", fileFormat = geom_format
#      );
  # Export the results with updated Transformer functions (09.08.17)
  ExportResultSet(
      resultSet, prefix = "V-{0}".format(vacancy), workingDirectory = r"VacancySupercells", fileFormat = geom_format
      );

# Generating antisite if requested by user
if (antisite_in != "no_antisite"):
   substitutions = [(antisite_out, antisite_in)];
   # Use the AtomicSubstitutions convenience function to generate the defective structures.
   _, resultSet = AtomicSubstitutions(
    supercell, substitutions, tolerance = tolerance
    );
   # Export the results.
   ExportResultSet(
      resultSet, prefix = "{0}-{1}".format(antisite_in, antisite_out), workingDirectory = r"AntisiteSupercells", fileFormat = geom_format
      );

## 6. Organise outputs and setup calculations for charge neutral defects

Below is a bash script which organises outputs from Transformer for charge neutral defect structures. It then copies and edits control.in files and job submission scripts in each defect directory. Ensure you have a control.in and submission script in the same directory as your script as outlined in step 4 and add the name of your submission script to the bash script below.
The script can be run from here (shift+enter the cell below).

Outputs are organised into 'VacancySupercells' and 'AntisiteSupercells', then into defect type (e.g. 'V-Cu' for a Cu vacancy) and then within each directory here you have a perfect supercell reference and a directory for each different possible defect supercell, distinguished by its space group.

**Bug note:** Script failed for enargite V-S which had two defects with same spacegroup:
x V-S_002_S63Cu48As16/S63Cu48As16_SG-6_0001.geometry.in
x V-S_002_S63Cu48As16/S63Cu48As16_SG-6_0002.geometry.in

In [ ]:
%%bash

### Ensure you have a control.in and a submission script in the same directory as this bash script
#### Set JobNamePlaceholder as the job name in your submission script
#### Include '#charge ChargePlaceholder' in your control.in
#### Enter the name of your submission script at the top of this script

#################################################################################################

# ENTER NAME OF SUBMISSION SCRIPT
SubmissionScriptName="submit.sh"

#################################################################################################

for defect in Vacancy Antisite
  do

  cd ${defect}Supercells

  # Taking defect-type to make directories
  ls * > files.dat
  cut -d_ -f1 files.dat > DirNames.dat
  # Remove duplicate directory names and cleanup
  awk '!seen[$0]++' DirNames.dat > UniqueDirNames.dat
  rm files.dat DirNames.dat

  # Making directories for each defect type and moving tar files for structures to correct directory
  while read dirname others; do
      mkdir "$dirname"
      mv ${dirname}*.gz $dirname
  done < UniqueDirNames.dat
  
    # Extracting structure for perfect reference and moving to its own directory
  while read dirname others; do
     cd $dirname
     tar -xvf ${dirname}_001*         # 001 directory is always perfect structure
     mkdir PerfectReference
     cp ${dirname}_001*/*geometry.in PerfectReference/perfect_geometry.in
     rm -r ${dirname}_001*
     cp ../../control.in PerfectReference
     cp ../../${SubmissionScriptName} PerfectReference
     # Name job in submission script
     sed -i -e "s/JobNamePlaceholder/PerfectSupercell/g" PerfectReference/${SubmissionScriptName}
     rm PerfectReference/${SubmissionScriptName}-e  # cleanup
     cd ..
  done < UniqueDirNames.dat

  # Extracting structures for point defect
  while read dirname others; do
     cd $dirname
     tar -xvf ${dirname}_002*         # 002 directory contains structures for point defect
     # Extracting spacegroup from filename to use as directory
     cd ${dirname}_002*
     ls * > filename_spacegroups.dat
     awk '{print $NF}' FS=- filename_spacegroups.dat > filenames_sg_trimmed.dat  # Takes everything after hyphen
     cut -d_ -f1 filenames_sg_trimmed.dat > filenames_sg_trimmed2.dat  # Deletes everything after _


     while read spacegroup others; do
       mkdir ../DefectSpacegroup${spacegroup}
       mv *SG-${spacegroup}_* ../DefectSpacegroup${spacegroup}/geometry.in
       cp ../../../control.in ../DefectSpacegroup${spacegroup}
       cp ../../../${SubmissionScriptName} ../DefectSpacegroup${spacegroup}
       # Name job in submission script
       sed -i -e "s/JobNamePlaceholder/${dirname}_SG_${spacegroup}_neutral/g" ../DefectSpacegroup${spacegroup}/${SubmissionScriptName}
       rm ../DefectSpacegroup${spacegroup}/${SubmissionScriptName}-e  # cleanup
     done < filenames_sg_trimmed2.dat
     rm filenames_sg_trimmed.dat filenames_sg_trimmed2.dat  # cleanup

     cd ..
     rm -r ${dirname}_002*
     cd ..
  done < UniqueDirNames.dat
  
    rm UniqueDirNames.dat  # cleanup
  cd ..

done

## 7. Identify all charge states for your defects and set up calculations

The next script then identifies all possible charge states for each defect and generates directories with inputs for these calculations by uncommenting and editing the `charge` keyword in the control.in file.

For this, please input the oxidation state of each species in your compound into the bash script below, which can be run from here (shift+enter on the cell below).

**Script ideas (underconstruction)**
Setting up charge states
0. Loop over directories for each defect mv current contents to 'neutral' dir
1. Calculate possible charge states for each species type (from user inputs)
2. mkdir charged --> +2, +1, etc.
3. cp geometry.in, control.in, submission script, defect_info.txt
4. Modify defect_info.txt with charge state for all charged defects
5. Edit charge placeholder in control.in, edit placeholder in submission script (sed neutral with q=...)

New plan:
- list vacancies
- list antisites
- loop over V-${vacancy}, 
  - calc possible charge states
  - cd each Defect* dir, mv current contents to 'neutral' dir, mkdir for each possible charge state, sed '# charge' in control.in and 'neutral' in submission script

In [ ]:
%%bash

##### UNDER CONSTRUCTION #####

# Species oxidation states for enargite, Cu3AsS4
OxState1 = 1 # Cu
OxState2 = 5 # As
OxState3 = -2 # S
OxState4 = 0 # No fourth component in enargite
species1 = "Cu"
species2 = "As"
species3 = "S"
species4 = "None" # No fourth component in enargite

# Calculating all possible charge states for vacancies
charge_max = -{$OxState}
charges = range(0,charge_max)
mkdir V-${species}/charged/$charges

# Setting up directories
sed '#charge ChargePlaceholder'/ 'charge $i'
sed 'JobNamePlaceholder'/ 'V_{$species_out}_q_{$i}'


# Calculating all possible charge states for antisites
species1 = species_out
for species_in in species2 species3 species4
  if species_in != 0 # checking this component exists in the compound, i.e not forth component in a ternary
    charge_max = {$OxState2}-{$OxState1}
    range(0,charge_max)

# Setting up directories
sed '#charge ChargePlaceholder'/ 'charge $i'
sed 'JobNamePlaceholder'/ '{$species_in}_{$species_out}_q_{$i}'


## 8. Optional step to distort the geometry around the defect site
This may be necessary for highly symmetric crystals to prevent the geometry of the defective supercell from finding a local (but not global) minima due to symmetry constraints.

Use the script below to distort your geometry.in files for sites neighbouring the defect site. This script uses the defect location information from defect_info.txt to determine which sites to displace. Adjust the loop over defect calculation directories as you see fit to decide which defect supercells you wish to distort.

** Under construction **

## 9. Finite-size corrections to defect supercells

** Under construction **

Apply image-charge correction and potential alignment with Lany-Zunger correction scheme.